# Decrypting Messages with MCMC

In [3]:
import warnings
from collections import Counter
import numpy as np
import networkx as nx
from IPython.display import display, Markdown

warnings.filterwarnings('ignore')

In [4]:
with open('symbols.txt', 'r', encoding='utf-16') as f:
    symbols = f.read().split('\n')

with open('war-and-peace.txt', 'r') as f:
    war_and_peace = f.read()
    war_and_peace = ' '.join(war_and_peace.lower().split())

In [5]:
unrecognized_symbols = set(war_and_peace).difference(symbols)
for uc in unrecognized_symbols:
    war_and_peace = war_and_peace.replace(uc, ' ')

war_and_peace = ' '.join(war_and_peace.split())

## 5 (a)

In [6]:
class MarkovChain:

    def __init__(self, symbols, text):

        self.symbols = symbols
        self.symbols_to_idx = {s: i for i, s in enumerate(symbols)}
        
        transition_counts = np.zeros((len(symbols), len(symbols))) + 1   # Laplace smoothing: setting uniform prior, avoids transition probabilities 0
        for beta, alpha in zip(text[:-1], text[1:]):
            transition_counts[self.symbols_to_idx[beta], self.symbols_to_idx[alpha]] += 1
        self.tm = transition_counts / transition_counts.sum(axis=1, keepdims=True)

        eig_vals, eig_vecs = np.linalg.eig(self.tm.T)
        p_inv = eig_vecs[:, np.where(np.isclose(eig_vals, 1))[0][0]]
        self.p_inv = (p_inv / np.sum(p_inv)).astype(float)

    def transition_prob(self, beta, alpha):

        return self.tm[self.symbols_to_idx[beta], self.symbols_to_idx[alpha]]

    def limiting_prob(self, gamma):

        return self.p_inv[self.symbols_to_idx[gamma]]

In [7]:
mc = MarkovChain(symbols, war_and_peace)

display(Markdown(
    fr"$p\left(s_i=b|s_{{i-1}}=a\right) = \psi\left(b, a\right) = {mc.transition_prob('a', 'b'):.6f}$"
))

$p\left(s_i=b|s_{i-1}=a\right) = \psi\left(b, a\right) = 0.017064$

In [8]:
assert np.allclose(mc.tm.T @ mc.p_inv, mc.p_inv))
assert np.isclose(np.sum(mc.p_inv), 1)

display(Markdown(
    '\n'.join([
        r'| $\gamma$ |' + ''.join([f" {symbol} |" for symbol in symbols]),
        '| :---: |' + ' :---: |'*len(symbols),
        r'| $\psi\left(\gamma\right)$ |' + ''.join([f' {p:.6f} |' for p in mc.p_inv]),
    ])
))

| $\gamma$ | = |   | - | , | ; | : | ! | ? | / | . | ' | " | ( | ) | [ | ] | * | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| $\psi\left(\gamma\right)$ | 0.000017 | 0.183298 | 0.000590 | 0.012503 | 0.000375 | 0.000332 | 0.001246 | 0.000998 | 0.000019 | 0.009682 | 0.000019 | 0.000023 | 0.000225 | 0.000225 | 0.000017 | 0.000017 | 0.000107 | 0.000070 | 0.000139 | 0.000062 | 0.000035 | 0.000024 | 0.000033 | 0.000033 | 0.000028 | 0.000077 | 0.000027 | 0.063472 | 0.010865 | 0.019308 | 0.037046 | 0.098187 | 0.017200 | 0.016084 | 0.052425 | 0.053931 | 0.000823 | 0.006414 | 0.030231 | 0.019312 | 0.057673 | 0.059525 | 0.014266 | 0.000746 | 0.046485 | 0.051008 | 0.070901 | 0.020176 | 0.008494 | 0.018558 | 0.001389 | 0.014494 | 0.000764 |

## 5 (d)

In [9]:
with open('message.txt', 'r', encoding='utf-16') as f:
    message = f.read()

In [10]:
def log_likelihood(sigma):

    sigma_inv = {e: d for d, e in sigma.items()}
    decrypted_text = ''.join([sigma_inv[e] for e in message])

    l = np.log(mc.limiting_prob(decrypted_text[0]))
    for beta, alpha in zip(decrypted_text[:-1], decrypted_text[1:]):
        transition_prob = mc.transition_prob(beta, alpha)
        l += np.log(transition_prob+1e-20)
        
    return l

In [11]:
war_and_peace_cntr = dict(Counter(war_and_peace))
message_cntr = Counter(message)
message_cntr = {k: message_cntr.get(k, 0) for k in war_and_peace_cntr}

In [19]:
permutation = {d: e for d, e in zip(sorted(war_and_peace_cntr.keys(), key=lambda x: war_and_peace_cntr[x]), sorted(message_cntr.keys(), key=lambda x: message_cntr[x]))}
updates = 0

display(Markdown(
    r'##### *Number of Updates / Number of Iterations: Decrypted Text*'
))

for i in range(1, 50001):

    s, s_ = np.random.choice(symbols, 2, replace=False)
    proposal = permutation.copy()
    proposal[s], proposal[s_] = proposal[s_], proposal[s]
    log_likelihood_diff = log_likelihood(proposal) - log_likelihood(permutation)

    if np.random.uniform() <= np.exp(log_likelihood_diff):
        permutation = proposal.copy()
        updates += 1

    if i%500 == 0:
        permutation_inv = {e: d for d, e in permutation.items()}
        print(f"{(str(updates)+'/'+str(i)).rjust(10, ' ')}: {''.join([permutation_inv[e] for e in message[:60]])}")

##### *Number of Updates / Number of Iterations: Decrypted Text*

    98/500: or bw wuaryed irn bude masredipse weidt bw gilfed yime be tu
  195/1000: or bf fuirden ary bune milrenaple feans bf gatwen dame be su
  276/1500: on my yuinder anf mure kilnerable years my gatwer dake me su
  355/2000: on my yuinfer and mure kilnerable years my gatwer fake me su
  432/2500: on my yuinger and mure kilnerable years my father gake me su
  519/3000: on my yuinger and mure kilnerable years my father gake me su
  603/3500: on my yuinger and mure vilnerable years my father gave me su
  686/4000: on my yuinger and mure vilnerable years my father gave me su
  759/4500: on my yuinger and mure vilnerable years my father gave me su
  829/5000: on my yuinger and mure vilnerable years my father gave me su
  913/5500: on my yuinger and mure vilnerable years my father gave me su
  995/6000: on my yuinger and mure vilnerable years my father gave me su
 1067/6500: on my yuinger and mure vilnerable years my father gave me su
 1157/7000: on my yuinger and mure vilnerable years

## 5 (e)

In [ ]:
edges = list()
for beta in symbols:
    for alpha in symbols:
        if mc.transition_prob(beta, alpha) > 0:
            edges.append(tuple((beta, alpha)))

G = nx.DiGraph()
G.add_edges_from(edges)
print(f'Number of strongly connected components = {len(list(nx.strongly_connected_components(G)))}')

Number of strongly connected components = 1


In [28]:
self_transition_probs = np.diag(mc.tm)
chars = [symbols[i] for i in np.where(self_transition_probs > 0)[0]]

print('Characters that can transition to themselves are ' + ', '.join(chars))

Characters that can transition to themselves are -, ., ', *, 0, 1, 2, 6, 8, a, b, c, d, e, f, g, h, i, l, m, n, o, p, r, s, t, v, w, x, z
